# Distance Measure for Heterogeneous Tabular Data

There has been many attempts to define a reliable distance measure for heterogeneous tabulr data (e.g., categorical, binary, and numerical features). The most notable is the Gower distance, which is a generalization of the Manhattan distance. But it is paticularly unstable when only a few features of a type are present in the data.

In the following we define a new distance metric that builds on the previous work of Gower and others. The new distance metric is based on the following principles:
1. Like the Gower, and Estabrook-rogers etc. the new distance metric is a collection of distance measures for each attribute. 
2. The distance for categorical features are based on the Hamming distance.
3. The distance for numerical and ordinal features are based on the normalised Manhattan distance.
4. Each attribute-wise distance measure is weighted by the probability of accidentally selecting the value for the attribute of the source point.

In [21]:
# Load datasets
import uci_dataset as dataset
import seaborn as sns
from prepare_data import _clean_up_data, uci_dataset_id_import

ID_nums = [ 44, 53, 174, 42, 95, 342, 212, 763, 39, 52, 292, 887, 537, 451, 16, 176, 43, 545, 149, 151, 110, 379, 186, 267, 146, 147, 80, 563, 107, 186]
ID_diff = [1, 257, 12, 14, 143, 15, 17, 145, 144, 22, 544, 161, 419, 73, 336, 83, 856, 857, 90, 101, 105, 244]

df = uci_dataset_id_import(12, silent_import=False)
# df = dataset.load_parkinson()
# df = sns.load_dataset('titanic')

print("Number of rows:", df.shape[0])

# _clean_up_data(df, 'class')
df.head()

Dataset Name: Balance Scale
Number of rows: 625


,right-distance,right-weight,left-distance,left-weight,class
0,1,1,1,1,B
1,2,1,1,1,R
2,3,1,1,1,R
3,4,1,1,1,R
4,5,1,1,1,R


In [1]:
import pandas as pd
df = pd.read_csv("01_knn_cls_results.csv")

# grab the part ot he dataframe with balanced type
# df = df[df['type']=='most_cats']

# take the dataframe, make accuracy, precision and recall columns into long format
df = df.melt(id_vars=['df_name', 'method'], value_vars=['accuracy', 'precision', 'recall', 'f1_score'],
             var_name='metric', value_name='value')
df
# df.rename({'df_name'})

,df_name,method,metric,value
0,autism,L2,accuracy,0.879433
1,autism,L2_OHE,accuracy,0.865248
2,autism,REX,accuracy,0.964539
3,autism,Gower,accuracy,0.964539
4,balance_scale,L2,accuracy,0.856000
...,...,...,...,...
459,thoracic_surgery,Gower,f1_score,0.782587
460,wisconsin_bc,L2,f1_score,0.946764
461,wisconsin_bc,L2_OHE,f1_score,0.946764
462,wisconsin_bc,REX,f1_score,0.762406


In [2]:
from critdd import Diagrams # Diagrams is the 2D version of Diagram
import numpy as np

# construct a sequence of CD diagrams
treatment_names = df["method"].unique()
diagram_names = df["metric"].unique()
Xs = [] # collect an (n,k)-shaped matrix for each diagram
for n in diagram_names:
    diagram_df = df[df.metric == n].pivot(
        index = "df_name",
        columns = "method",
        values = "value"
    )[treatment_names] # ensure a fixed order of treatments
    Xs.append(diagram_df.to_numpy())
two_dimensional_diagram = Diagrams(
    np.stack(Xs),
    diagram_names = diagram_names,
    treatment_names = treatment_names,
    maximize_outcome = True
)

# customize the style of the plot and export to PDF
two_dimensional_diagram.to_file(
    "2d_example.pdf",
    preamble = "\n".join([ # colors are defined before \begin{document}
        "\\definecolor{color1}{HTML}{84B818}",
        "\\definecolor{color2}{HTML}{D18B12}",
        "\\definecolor{color3}{HTML}{1BB5B5}",
        "\\definecolor{color4}{HTML}{F85A3E}",
        "\\definecolor{color5}{HTML}{4B6CFC}",
        "\\definecolor{color6}{HTML}{F8A300}",
        "\\definecolor{color7}{HTML}{FF69B4}",
    ]),
    axis_options = { # style the plot
        "cycle list": ",".join([ # define the markers for treatments
            "{color1,mark=*}",
            "{color2,mark=diamond*}",
            "{color3,mark=triangle,semithick}",
            "{color4,mark=square,semithick}",
            "{color5,mark=pentagon,semithick}",
            "{color6,mark=star,semithick}",
            "{color7,mark=oplus,semithick}",
        ]),
        "width": "\\axisdefaultwidth",
        "height": "0.75*\\axisdefaultheight",
        "title": "critdd"
    },
)

### Old stuff

In [101]:
# split data
from sklearn.model_selection import train_test_split

# encode categorical features
from sklearn.preprocessing import OrdinalEncoder

def encode_categorical_features(df, cat_features):
    encoder = OrdinalEncoder()
    df[cat_features] = encoder.fit_transform(df[cat_features])
    return df

def get_categorical_features(df, unique_threshold=10):
    cat_features = df.select_dtypes(include=['object']).columns.tolist()

    for col in df.select_dtypes(include=['int', 'float']).columns:
        if df[col].nunique() <= unique_threshold:
            cat_features.append(col)
    return cat_features

def split_data(df, test_size=0.2, random_state=42):  
    # Split the data into train and test sets
    train, test = train_test_split(df, test_size=test_size, random_state=random_state)
    return train, test

def process_data(df, class_col='class'):
    cat_features = get_categorical_features(df)
    print("Categorical features:", cat_features)
    df = encode_categorical_features(df, cat_features)
    
    df = df.dropna()
    df = df[[class_col] + [col for col in df.columns if col != class_col]]
    cat_features.remove(class_col)

    train, test = split_data(df)
    return train, test, cat_features

train, test, cat_cols = process_data(d3, 'NSP')

print("Train set shape:", train.shape)
print("Test set shape:", test.shape)

Categorical features: ['NSP', 'DS', 'Nzeros', 'Tendency', 'CLASS']
Train set shape: (1700, 23)
Test set shape: (426, 23)


In [102]:
from sklearn.metrics import accuracy_score, f1_score

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train.iloc[:, 1:], train.iloc[:,0])
preds = neigh.predict(test.iloc[:, 1:])

f1_score(test.iloc[:,0], preds, average='macro')

0.8110040156190678

In [103]:
# use gower distance
import gower

def gower_knn(train, test, k=3):
    # Create a KNN classifier
    knn = KNeighborsClassifier(n_neighbors=k, metric='precomputed')
    
    # Compute the Gower distance matrix
    gower_train = gower.gower_matrix(train.iloc[:, 1:], train.iloc[:, 1:])
    gower_test = gower.gower_matrix(test.iloc[:, 1:], train.iloc[:, 1:])
    print(gower_test.shape)
    
    # Fit the model
    knn.fit(gower_train, train.iloc[:, 0])
    
    # Predict the labels for the test set
    predictions = knn.predict(gower_test)
    return predictions

preds = gower_knn(train, test, k=3)

f1_score(test.iloc[:,0], preds, average='macro')

(426, 1700)


0.9043687560648586

In [58]:
import numpy as np
import pandas as pd

def _scott_ref_rule(samples):
    """Function for doing the Scott reference rule to calcualte number of bins needed to 
    represent the nummerical values.
    
    Args:
        samples (array-like) : The data to be binned.
    
    Returns:
        array : bin edges
    
    Example:
        >>> _scott_ref_rule([1,2,3,4,5])
        array([1., 2., 3., 4., 5.])
    """
    std = np.std(samples)
    n = len(samples)
    bin_width = np.ceil(n**(1/3) * std / (3.5 * (np.percentile(samples, 75) - np.percentile(samples, 25)))).astype(int)

    min_edge = min(samples); max_edge = max(samples)
    N = min(abs(int((max_edge-min_edge)/bin_width)),10000); Nplus1 = N + 1
    return np.linspace(min_edge, max_edge, Nplus1)

def lautrup_distance_matrix(df_x, df_y=None, cat_features=None):
    """
    Compute the Lautrup distance matrix between two dataframes.
    """

    # join the datasets
    if df_y is None:
        df_y = df_x
    df = pd.concat((df_x,df_y), axis=0)

    num_features = [col for col in df.columns if col not in cat_features]

    normalisation, feat_range = {}, {}
    for col in num_features:
        if np.issubdtype(df[col].dtype, np.floating):
            normalisation[col] = 1-1/(max(len(_scott_ref_rule(df[col])),2)-1)
        else:
            normalisation[col] = 1-1/df[col].nunique()

        feat_range[col] = df[col].max()-df[col].min()

    for col in cat_features:
        normalisation[col] = 1-1/df[col].nunique()

    # create the distance matrix
    dist_matrix = np.zeros((len(df_x), len(df_y)))

    for i, row in enumerate(df_x.iterrows()):
        for j, col in enumerate(df_y.iterrows()):
            for col_name in num_features:
                dist_matrix[i,j] += normalisation[col_name]*np.abs(row[1][col_name] - col[1][col_name]) / feat_range[col_name]
            for col_name in cat_features:
                if row[1][col_name] != col[1][col_name]:
                    dist_matrix[i,j] += normalisation[col_name]
    return dist_matrix

def lautrup_knn(train, test, k=3, cat_features=None):
    # Create a KNN classifier
    knn = KNeighborsClassifier(n_neighbors=k, metric='precomputed')
    
    # Compute the Gower distance matrix
    _train = lautrup_distance_matrix(train.iloc[:, 1:], train.iloc[:, 1:], cat_features)
    _test = lautrup_distance_matrix(test.iloc[:, 1:], train.iloc[:, 1:], cat_features)
    print(_test.shape)
    
    # Fit the model
    knn.fit(_train, train.iloc[:, 0])
    
    # Predict the labels for the test set
    predictions = knn.predict(_test)
    return predictions

cat_vars = ['menopause', 'node-caps', 'breast', 'breast-quad', 'irradiat']
preds = lautrup_knn(train, test, k=3, cat_features=cat_vars)

f1_score(test.iloc[:,0], preds, average='macro')

C:\Users\lautrup\AppData\Local\Temp\1\ipykernel_13092\4047891434.py:20: RuntimeWarning: divide by zero encountered in scalar divide
  bin_width = np.ceil(n**(1/3) * std / (3.5 * (np.percentile(samples, 75) - np.percentile(samples, 25)))).astype(int)
C:\Users\lautrup\AppData\Local\Temp\1\ipykernel_13092\4047891434.py:20: RuntimeWarning: invalid value encountered in cast
  bin_width = np.ceil(n**(1/3) * std / (3.5 * (np.percentile(samples, 75) - np.percentile(samples, 25)))).astype(int)


(56, 221)


0.5579559617781471

In [104]:
import numpy as np
import pandas as pd

from numpy import ndarray
from pandas import Series

def _scott_ref_rule(samples):
    """Function for doing the Scott reference rule to calcualte number of bins needed to 
    represent the nummerical values.
    
    Args:
        samples (array-like) : The data to be binned.
    
    Returns:
        array : bin edges
    
    Example:
        >>> _scott_ref_rule([1,2,3,4,5])
        array([1., 2., 3., 4., 5.])
    """
    std = np.std(samples)
    n = len(samples)
    bin_width = np.ceil(n**(1/3) * std / (3.5 * (np.percentile(samples, 75) - np.percentile(samples, 25)))).astype(int)

    min_edge = min(samples); max_edge = max(samples)
    N = min(abs(int((max_edge-min_edge)/bin_width)),10000); Nplus1 = N + 1
    return np.linspace(min_edge, max_edge, Nplus1)

def prob_reroll_cat(counts: Series, target: int):
    """Calculate the probability of rolling a categorical variable into the target variable."""
    return counts.get(target, 0)

def prob_reroll_num(histogram: ndarray, binning: ndarray,  target: float):
    """Calculate the probability of rolling a numerical variable into the target variable."""
    # Get the counts of each bin in the variable
    
    bin_index = np.digitize(target, binning) - 1
    if bin_index < 0 or bin_index >= len(histogram):
        return 0
    prob = histogram[bin_index] / sum(histogram)
    return prob

def rerollers_distance_matrix(df_x, df_y=None, cat_features=None):
    """
    Compute the rerollers distance matrix between two dataframes.
    (the chance that randomly resampling the variable will make it into the target variable)
    """

    # join the datasets
    if df_y is None:
        df_y = df_x
    df = pd.concat((df_x,df_y), axis=0)

    num_features = [col for col in df.columns if col not in cat_features]

    bin_range, histogram, feat_range = {}, {}, {}
    for col in num_features:
        if np.issubdtype(df[col].dtype, np.floating):
            bin_range[col] = _scott_ref_rule(df[col])
            histogram[col] = np.histogram(df[col], bins=bin_range[col])[0]
        else:
            bin_range[col] = np.arange(df[col].nunique()+1)
            histogram[col] = np.histogram(df[col], bins=bin_range[col])[0]
            
        feat_range[col] = df[col].max()-df[col].min()
    
    cat_counts = {}
    for col in cat_features:
        cat_counts[col] = df[col].value_counts(normalize=True)

    # create the distance matrix
    dist_matrix = np.zeros((len(df_x), len(df_y)))

    for i, row in enumerate(df_x.iterrows()):
        for j, col in enumerate(df_y.iterrows()):
            for col_name in num_features:
                dist_matrix[i,j] += (1-prob_reroll_num(histogram[col_name], bin_range[col_name], j))*np.abs(row[1][col_name] - col[1][col_name]) / feat_range[col_name]
            for col_name in cat_features:
                if row[1][col_name] != col[1][col_name]:
                    dist_matrix[i,j] += (1-prob_reroll_cat(cat_counts[col_name], j))
    return dist_matrix

def reroll_knn(train, test, k=3, cat_features=None):
    # Create a KNN classifier
    knn = KNeighborsClassifier(n_neighbors=k, metric='precomputed')
    
    # Compute the Gower distance matrix
    _train = rerollers_distance_matrix(train.iloc[:, 1:], train.iloc[:, 1:], cat_features)
    _test = rerollers_distance_matrix(test.iloc[:, 1:], train.iloc[:, 1:], cat_features)
    print(_test.shape)
    
    # Fit the model
    knn.fit(_train, train.iloc[:, 0])
    
    # Predict the labels for the test set
    predictions = knn.predict(_test)
    return predictions

preds = reroll_knn(train, test, k=3, cat_features=cat_cols)

f1_score(test.iloc[:,0], preds, average='macro')

C:\Users\lautrup\AppData\Local\Temp\1\ipykernel_13092\3680320830.py:23: RuntimeWarning: divide by zero encountered in scalar divide
  bin_width = np.ceil(n**(1/3) * std / (3.5 * (np.percentile(samples, 75) - np.percentile(samples, 25)))).astype(int)
C:\Users\lautrup\AppData\Local\Temp\1\ipykernel_13092\3680320830.py:23: RuntimeWarning: invalid value encountered in cast
  bin_width = np.ceil(n**(1/3) * std / (3.5 * (np.percentile(samples, 75) - np.percentile(samples, 25)))).astype(int)


(426, 1700)


0.9545681900554648